# NLP - Classificação de texto com NLTK e Naive Bayes
## Classificação negativa ou positiva de *reviews* de filmes
Os registros estão divididos em pastas de acordo com sua classe, neg e pos. O projeto a seguir tem como objetivo ler os arquivos com avaliações de filmes, fazer o pré-processamento para o modelo do algoritmo utilizado, treinar e depois validar o mesmo.
#### Por Leonardo Bonifácio

In [1]:
from nltk.corpus import movie_reviews
import random
import nltk
import pickle

## Pré-processamento

#### Para leitura dos arquivos, é criada uma lista onde cada arquivo com sua respectiva avaliação e classe é armazenada em uma tupla.

In [2]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [3]:
documents[0]

(['plot',
  ':',
  'two',
  'teen',
  'couples',
  'go',
  'to',
  'a',
  'church',
  'party',
  ',',
  'drink',
  'and',
  'then',
  'drive',
  '.',
  'they',
  'get',
  'into',
  'an',
  'accident',
  '.',
  'one',
  'of',
  'the',
  'guys',
  'dies',
  ',',
  'but',
  'his',
  'girlfriend',
  'continues',
  'to',
  'see',
  'him',
  'in',
  'her',
  'life',
  ',',
  'and',
  'has',
  'nightmares',
  '.',
  'what',
  "'",
  's',
  'the',
  'deal',
  '?',
  'watch',
  'the',
  'movie',
  'and',
  '"',
  'sorta',
  '"',
  'find',
  'out',
  '.',
  '.',
  '.',
  'critique',
  ':',
  'a',
  'mind',
  '-',
  'fuck',
  'movie',
  'for',
  'the',
  'teen',
  'generation',
  'that',
  'touches',
  'on',
  'a',
  'very',
  'cool',
  'idea',
  ',',
  'but',
  'presents',
  'it',
  'in',
  'a',
  'very',
  'bad',
  'package',
  '.',
  'which',
  'is',
  'what',
  'makes',
  'this',
  'review',
  'an',
  'even',
  'harder',
  'one',
  'to',
  'write',
  ',',
  'since',
  'i',
  'generally',
  'a

#### Preparação das palavras para treinamento

A lista all_words contém todas as palavras de todas as avaliações caso elas não sejam stopwords e caso sejam apenas letras(o que retira pontuação)

In [4]:
stopwords_nltk = nltk.corpus.stopwords.words('english') # Stopwords para o idioma inglês

all_words = [w.lower() for w in movie_reviews.words() if w not in stopwords_nltk if w.isalpha()]

Depois ela é organizada de acordo com sua frequência de forma crescente, onde o início da lista contém os menos frequentes, e no final, os mais frequentes.

In [5]:
all_words = nltk.FreqDist(all_words)
print(all_words.most_common(20))

[('film', 9517), ('one', 5852), ('movie', 5771), ('like', 3690), ('even', 2565), ('good', 2411), ('time', 2411), ('story', 2169), ('would', 2109), ('much', 2049), ('character', 2020), ('also', 1967), ('get', 1949), ('two', 1911), ('well', 1906), ('characters', 1859), ('first', 1836), ('see', 1749), ('way', 1693), ('make', 1642)]


In [6]:
print(len(all_words))

38738


Dentre as palavras mais comuns, muitas não tem relevância no treinamento do modelo, visto que são genéricas e seu significado não contribui para a classificação. Sendo assim, elas são divididas somente com as 3000 palavras menos comuns, que retiram as palavras desnecessárias e, ao mesmo tempo, contém as palavras que podem trazer relevância para o modelo.

In [7]:
word_features = list(all_words.keys())[:3000]

#### Extração de características e criação da tabela

Segundo o padrão para treinamento utilizando Naive Bayes, as palavras mais frequentes e relevantes devem estar numa lista onde a base a ser treinada vai ser comparada, palavra por palavra, retornando True ou False sobre estar contida nessa lista.

In [8]:
def find_features(document):
	words = set(document)
	features = {}

	for w in word_features:
		features[w] = (w in words)

	return features

A base é obtida varrendo cada avaliação contida nos documentos, e retorna uma lista de tuplas com o dicionário de comparação de palavras explicado acima e sua respectiva categoria.

In [9]:
feature_set = [(find_features(rev), category) for (rev, category) in documents]

In [10]:
print(feature_set[0])

({'plot': True, 'two': True, 'teen': True, 'couples': True, 'go': True, 'church': True, 'party': True, 'drink': True, 'drive': True, 'get': True, 'accident': True, 'one': True, 'guys': True, 'dies': True, 'girlfriend': True, 'continues': True, 'see': True, 'life': True, 'nightmares': True, 'deal': True, 'watch': True, 'movie': True, 'sorta': True, 'find': True, 'critique': True, 'mind': True, 'fuck': True, 'generation': True, 'touches': True, 'cool': True, 'idea': True, 'presents': True, 'bad': True, 'package': True, 'makes': True, 'review': True, 'even': True, 'harder': True, 'write': True, 'since': True, 'generally': True, 'applaud': True, 'films': True, 'attempt': True, 'break': True, 'mold': True, 'mess': True, 'head': True, 'lost': True, 'highway': True, 'memento': True, 'good': True, 'ways': True, 'making': True, 'types': True, 'folks': True, 'snag': True, 'correctly': True, 'seem': True, 'taken': True, 'pretty': True, 'neat': True, 'concept': True, 'executed': True, 'terribly': 

## Treinamento e validação do modelo

Para dividir as bases de teste e treinamento, a função abaixo seleciona registros aleatórios de uma porcentagem n para serem utilizados no teste e o restante para treinamento

In [11]:
def baseGenerator(dataset, n):
    perc = round(n*len(dataset))
    random_index = random.sample(range(0,len(dataset)-1), perc)
    testing_set = []
    training_set = []
    
    for i in range(perc):
        testing_set.append(dataset[random_index[i]])
    for j in range(len(dataset)):
        if(j not in random_index):
            training_set.append(dataset[j])
            
    return  training_set, testing_set

In [12]:
training_set, testing_set = baseGenerator(feature_set, 0.15) # 15% utilizados para testar
print('Tamanho da base de treinamento:',len(training_set))
print('Tamanho da base de teste:',len(testing_set))

Tamanho da base de treinamento: 1700
Tamanho da base de teste: 300


#### Treinamento e salvando modelo
Deve ser executado somente na primeira vez para treinar e salvar o modelo

In [13]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

save_classifier = open("naive_bayes_classifier.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

#### Carregando modelo e testando
Com o modelo já treinado e salvo, basta apenas carregá-lo e treinar utilizando a base de teste

In [14]:
classifier_f = open("naive_bayes_classifier.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

#### Resultados
Mostra a acurácia do modelo e as palavras mais relevantes para classificação.

In [15]:
accuracy = nltk.classify.accuracy(classifier, testing_set)

print("Acuracia do modelo = ", round(accuracy,4))
classifier.show_most_informative_features(10)

Acuracia do modelo =  0.7867
Most Informative Features
              schumacher = True              neg : pos    =     10.8 : 1.0
                   sucks = True              neg : pos    =      8.8 : 1.0
                 frances = True              pos : neg    =      8.5 : 1.0
                  annual = True              pos : neg    =      8.5 : 1.0
                 idiotic = True              neg : pos    =      7.7 : 1.0
                   waste = True              neg : pos    =      7.3 : 1.0
                  regard = True              pos : neg    =      7.1 : 1.0
                  crappy = True              neg : pos    =      6.9 : 1.0
                 martian = True              neg : pos    =      6.9 : 1.0
                    yawn = True              neg : pos    =      6.9 : 1.0


Como os resultados variam de acordo com a criação das bases, é necessário validar melhor o modelo. Isso pode ser feito calculando a acurácia média do modelo para n iterações:

In [16]:
def accuracyMean(classifier, dataset, n_iterations, n_test):
    accuracy_all = []
    accuracy_mean = 0
    accuracy_sum = 0
    
    for i in range(n_iterations):
        training_set, testing_set = baseGenerator(dataset, n_test)
        accuracy = nltk.classify.accuracy(classifier, testing_set)

        accuracy_all.append(accuracy)
        
        accuracy_sum += accuracy_all[i]
    accuracy_mean = accuracy_sum / n_iterations
    
    return round(accuracy_mean, 4)

A função recebe o classificador, o dataset, número de testes e a porcentagem para base de teste

In [17]:
accuracy_mean = accuracyMean(classifier, feature_set, 5, 0.15) 
print("A acurácia média foi de ", accuracy_mean)

A acurácia média foi de  0.894
